## Oil and Gas Production Forecast
### Part_3.
### 1.1 Import Librries

In [1]:
import pandas as pd
import numpy as np
import warnings

from  datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import category_encoders as ce
import matplotlib.pyplot as plt
import gc
%matplotlib inline
warnings.filterwarnings('ignore')

### 1.2 Read in data

In [2]:
fld = pd.read_csv('data/fld.csv')

In [3]:
df_1 = pd.read_csv('data/df_1.csv')
df_2 = pd.read_csv('data/df_2.csv')
df_3 = pd.read_csv('data/df_3.csv')
df = df_1.append(df_2).append(df_3)
del df_1, df_2, df_3
gc.collect()

11

In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df.head(4).append(df.tail(3))

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
0,31009019120000,1988,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
1,31009019120000,1989,"Sager, Howard & Judith",0.0,500.0,0.0,0.0,GD
2,31009019120000,1990,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
3,31009019120000,1991,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
121155,31029255050000,2017,U. S. Energy Development Corporation,0.0,2126.0,42.0,7.0,Gas Development
121156,31029255050000,2018,U. S. Energy Development Corporation,0.0,318.0,45.0,7.0,Gas Development
121157,31029255050000,2019,U. S. Energy Development Corporation,0.0,2436.0,0.0,12.0,Gas Development


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121158 entries, 0 to 121157
Data columns (total 8 columns):
API Well Number         121158 non-null int64
Year                    121158 non-null int64
Operator                121158 non-null object
OIL (Bbls)              102545 non-null float64
GAS (Mcf)               120678 non-null float64
WATER (Bbls)            116452 non-null float64
Months in Production    120252 non-null float64
Well Type               121158 non-null object
dtypes: float64(4), int64(2), object(2)
memory usage: 7.4+ MB


In [7]:
fld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 51 columns):
API Well Number                   4717 non-null int64
County Code                       4717 non-null int64
API Hole Number                   4717 non-null int64
Sidetrack                         4717 non-null int64
Completion                        4717 non-null int64
Well Name                         4717 non-null object
Company Name                      4689 non-null object
Operator Number                   4717 non-null int64
Well Type                         4717 non-null object
Map Symbol                        4717 non-null object
Well Status                       4717 non-null object
Status Date                       572 non-null object
Permit Application Date           4413 non-null object
Permit Issued Date                4455 non-null object
Date Spudded                      4629 non-null object
Date of Total Depth               4460 non-null object
Date Well Completed 

In [8]:
fld.sample(3)

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Well Name,Company Name,Operator Number,Well Type,Map Symbol,...,Permit Fee,Objective Formation,Depth Fee,Spacing,Spacing Acres,Integration,Hearing Date,Date Last Modified,DEC Database Link,Location 1
4585,31013252350000,13,25235,0,0,Fee 1,Pine Valley Central School District,899,GD,GW,...,0.0,Medina,0.0,Exempt from Title 5; variance needed from 6 NY...,NaN,No integration order required,NaN,2009-10-22T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.095131', 'needs_recoding': ..."
4030,31013221230000,13,22123,0,0,"Nickerson, J 1",Chautauqua Energy Drilling Inc.,2752,GD,GW,...,975.0,Medina,875.0,NaN,NaN,NaN,NaN,2014-12-19T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.51345', 'needs_recoding': F..."
4360,31013244430000,13,24443,0,0,Cheney 5-490,PPP Future Development Inc.,2740,GD,GW,...,2000.0,Medina,1900.0,Exempt from Title 5; complies with 6 NYCRR 553...,NaN,No integration order required,NaN,2018-07-20T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.182355', 'needs_recoding': ..."


### Data Cleaning

In [9]:
fld['Well Type'].unique()

array(['GD'], dtype=object)

In [10]:
fld['Map Symbol'].unique()

array(['GWP', 'GW', 'O'], dtype=object)

In [11]:
fld.describe()

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Operator Number,Region,"Proposed Depth, ft",Surface Longitude,Surface Latitude,Bottom Hole Longitude,Bottom Hole Latitude,"True Vertical Depth, ft","Measured Depth, ft","Kickoff, ft","Drilled Depth, ft","Elevation, ft",Permit Fee,Depth Fee,Spacing Acres
count,4.717000e+03,4717.000000,4717.000000,4717.0,4717.000000,4717.000000,4717.0,4716.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.0,4717.000000,4444.000000,4716.000000,4717.000000,22.000000
mean,3.101271e+13,12.544626,16953.453890,0.0,0.018868,2272.140767,9.0,2812.433418,-79.316538,42.258476,-79.316536,42.258484,3554.433326,3554.430570,0.0,3554.430570,1369.596985,611.036684,559.059360,53.965909
std,1.620988e+09,1.622432,6011.530453,0.0,0.146576,1527.626292,0.0,1690.720419,0.222828,0.143486,0.222827,0.143491,775.597958,775.597873,0.0,775.597873,345.668537,656.604968,612.319424,27.441319
min,3.100901e+13,9.000000,255.000000,0.0,0.000000,13.000000,9.0,0.000000,-79.761600,41.999480,-79.761600,41.999480,0.000000,0.000000,0.0,0.000000,3.000000,0.000000,0.000000,35.200000
25%,3.101311e+13,13.000000,13043.000000,0.0,0.000000,1192.000000,9.0,2100.000000,-79.509110,42.151120,-79.509110,42.151120,3116.000000,3116.000000,0.0,3116.000000,1260.000000,0.000000,0.000000,40.000000
50%,3.101316e+13,13.000000,17148.000000,0.0,0.000000,2544.000000,9.0,3520.000000,-79.293470,42.257070,-79.293470,42.257070,3708.000000,3708.000000,0.0,3708.000000,1461.000000,20.000000,0.000000,41.030000
75%,3.101319e+13,13.000000,20246.000000,0.0,0.000000,2544.000000,9.0,4095.000000,-79.139240,42.369260,-79.139240,42.369260,4166.000000,4166.000000,0.0,4166.000000,1610.000000,1225.000000,1125.000000,51.130000
max,3.102966e+13,29.000000,67321.000000,0.0,2.000000,9998.000000,9.0,6501.000000,-78.756970,42.693560,-78.756970,42.693560,6182.000000,6182.000000,0.0,6182.000000,2625.000000,11000.000000,10900.000000,146.800000


In [12]:
fld['Completion'].value_counts()

0    4635
1      75
2       7
Name: Completion, dtype: int64

In [13]:
# fld['Spacing Acres'].unique()

In [14]:
# fld['Location 1'].unique()

In [15]:
# fld['Surface Latitude'].unique()
fld['Original Well Type'].unique()

array(['NL', 'GW', 'GD', 'OD'], dtype=object)

In [16]:
# Drop Numerical columns which are not required for analysis
fld.drop(columns=['API Hole Number','Sidetrack','Permit Fee', 
                  'Depth Fee', 'Spacing Acres'], inplace=True)

In [17]:
# Drop Categorical columns which are not required for analysis
fld.drop(columns=['Financial Security','Location 1', 'Integration',
                  'Hearing Date', 'Date Last Modified', 'DEC Database Link',
                  'Spacing', 'State Lease','Well Name'], inplace=True)

In [18]:
fld["Status Date"].isna().sum()

4145

In [19]:
fld['Company Name'].nunique(), fld['Operator Number'].nunique()

(352, 353)

In [20]:
fld.drop(columns=['Status Date', 'Permit Application Date', 'Map Symbol',
                 'Permit Issued Date','Company Name'], inplace=True)

In [21]:
fld.Quad.unique()

array(['Cherry Creek', 'Collins Center', 'Gowanda', 'Steamburg',
       'Farnham', 'North Collins', 'Randolph', 'New Albion', 'Kennedy',
       'Ivory', 'Perrysburg', 'Cattaraugus', 'Brocton', 'Silver Creek',
       'Dunkirk', 'Forestville', 'Cassadaga', 'Ellery Center',
       'North Of Dunkirk', 'Gerry', 'Ripley', 'South Ripley', 'Westfield',
       'Chautauqua', 'Hartfield', 'Jamestown', 'Sherman', 'Panama',
       'Clymer', 'North Clymer', 'Hamlet', 'Lakewood', 'North East PA',
       'North East', 'Wattsburg, PA-NY', 'Sugar Grove, PA-NY',
       'Warren Pa', 'North Of Westfield', 'Youngsville PA', 'Langford',
       'Angola'], dtype=object)

In [22]:
fld['Quad Section'].unique()

array(['I', 'G', 'F', nan, 'C', 'D', 'H', 'B', 'A', 'E'], dtype=object)

In [23]:
fld['Quad Section'].isna().sum()

9

In [24]:
fld['Date Well Confidentiality Ends'].isna().sum()

3090

In [25]:
#fld['Confidentiality Code'].isna().sum(), 
fld['Confidentiality Code'].unique()

array(['Pre-1989 Well (N/A)', 'Released'], dtype=object)

In [26]:
del fld['Date Well Confidentiality Ends'], fld['Confidentiality Code'], fld['Town']

In [27]:
fld[['Date Spudded','Date of Total Depth','Date Well Completed', 'Date Well Plugged']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 4 columns):
Date Spudded           4629 non-null object
Date of Total Depth    4460 non-null object
Date Well Completed    4717 non-null object
Date Well Plugged      353 non-null object
dtypes: object(4)
memory usage: 147.5+ KB


In [28]:
del fld['Date Spudded'], fld['Date of Total Depth'],fld['Date Well Plugged']

In [29]:
fld.columns

Index(['API Well Number', 'County Code', 'Completion', 'Operator Number',
       'Well Type', 'Well Status ', 'Date Well Completed', 'Quad',
       'Quad Section', 'Producing Field', 'Producing Formation', 'Slant',
       'County', 'Region', 'Proposed Depth, ft', 'Surface Longitude',
       'Surface Latitude', 'Bottom Hole Longitude', 'Bottom Hole Latitude',
       'True Vertical Depth, ft', 'Measured Depth, ft', 'Kickoff, ft',
       'Drilled Depth, ft', 'Elevation, ft', 'Original Well Type',
       'Objective Formation'],
      dtype='object')

In [36]:
fld['Date Well Completed'] = pd.to_datetime(fld['Date Well Completed'])
fld ['Complition year'] = fld['Date Well Completed'].dt.year
fld ['Complition month'] = fld['Date Well Completed'].dt.month
fld ['Complition day'] = fld['Date Well Completed'].dt.day
del fld['Date Well Completed']
# calendar['week'] = calendar['date'].dt.week.astype(np.int8)

In [45]:
# fld.isna().sum()

In [42]:
fld['Objective Formation'].unique()

array([nan, 'Medina', 'Onondaga', 'Clinton', 'Bass Island', 'Glade',
       'Theresa', 'Whirlpool', 'Queenston'], dtype=object)

In [43]:
fld['Producing Formation'].unique()

array(['Medina', nan, 'Whirlpool', 'Onondaga', 'Grimsby', 'Camillus',
       'Upper Devonian', 'Unknown', 'Salina', 'Bass Island', 'Tully',
       'Upper Devonian Shale'], dtype=object)

In [44]:
# drop Ob`jective formation, and leave Producing formation as actual one
del fld['Objective Formation']

In [47]:
fld.isna().sum()

API Well Number              0
County Code                  0
Completion                   0
Operator Number              0
Well Type                    0
Well Status                  0
Quad                         0
Quad Section                 9
Producing Field              0
Producing Formation         15
Slant                        0
County                       0
Region                       0
Proposed Depth, ft           1
Surface Longitude            0
Surface Latitude             0
Bottom Hole Longitude        0
Bottom Hole Latitude         0
True Vertical Depth, ft      0
Measured Depth, ft           0
Kickoff, ft                  0
Drilled Depth, ft            0
Elevation, ft              273
Original Well Type           0
Complition year              0
Complition month             0
Complition day               0
dtype: int64